In [4]:
#import libraries
from tinyec import registry
import secrets
import time
import paho.mqtt.client as mqtt

def initApp():
    global BROKER, PORT, ENCRYPTED_TEXT_TOPIC, CLIENT_PUBLIC_KEY_TOPIC, CLOUD_PUBLIC_KEY_TOPIC, client
    BROKER = "localhost"
    PORT = 1883
    CLIENT_PUBLIC_KEY_TOPIC = "client_pubkey"
    CLOUD_PUBLIC_KEY_TOPIC = "cloud_pubkey"
    ENCRYPTED_TEXT_TOPIC = "encrypted_txt"
    client = mqtt.Client("localhost", "792792")
    client.connect(BROKER, PORT)

curve = registry.get_curve('brainpoolP256r1')

def compress_point(point):
    return hex(point.x) + hex(point.y % 2)[2:]


#--------------------------------------------------------------------
#   Generate CLIENT_PRIVATE_KEY and CLIENT_PUBLIC_KEY

client_privKey = secrets.randbelow(curve.field.n)
client_pubKey = client_privKey * curve.g

#--------------------------------------------------------------------

def shared_key(privKey, other_pubKey):
    sharedECCKey = other_pubKey * privKey
    return sharedECCKey

def ecc_encrypt(message, client_privKey, cloud_pubKey):
    # Convert the message to bytes
    message_bytes = message.encode()

    # Generate a random key pair for encryption
    encryptKey = shared_key(client_privKey, cloud_pubKey)

    # Encrypt the message using the shared ECC encryption key
    encrypted = bytearray()
    shared_key_hex = compress_point(encryptKey)  # Renamed variable
    for byte in message_bytes:
        encrypted_byte = byte ^ int(shared_key_hex, 16) % 256  # Limit to byte range
        encrypted.append(encrypted_byte)

    return encrypted


#--------------------------------------------------------------------
#   Connect to an MQTT brocker and subscribe to a topic to receive the cloud public key
def mqttReceiveDataTask():
    return client.subscribe(CLOUD_PUBLIC_KEY_TOPIC)

#--------------------------------------------------------------------
#   Connect to an MQTT brocker and publish a the encrypted message and the client public key
def mqttCommunicateDataTask(ciphertextPubKey,encrypted_message):
    client.publish(CLIENT_PUBLIC_KEY_TOPIC, str(ciphertextPubKey))
    client.publish(ENCRYPTED_TEXT_TOPIC, str(encrypted_message))


#--------------------------------------------------------------------
#   Here you can generate your file

def loopApp():
    while True:
        message = "Hello, World!"
        cloud_pubKey = mqttReceiveDataTask()
        encrypted_message = ecc_encrypt(message, client_privKey, cloud_pubKey)
        print("Encrypted:", encrypted_message)
        mqttCommunicateDataTask(client_pubKey,encrypted_message)
        time.sleep(5)

if __name__ == '__main__':
    initApp()
    try: 
        loopApp()
    except:
        KeyboardInterrupt

#   END OF CODE